In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from utils.utils import \
    time_synchronized,select_device, increment_path,\
    scale_coords,xyxy2xywh,non_max_suppression,split_for_trace_model,\
    driving_area_mask,lane_line_mask,plot_one_box,show_seg_result,\
    AverageMeter,\
    LoadImages,\
    letterbox

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

In [3]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)



## Test with semantic segmentation

In [7]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    image.convert(carla.ColorConverter.CityScapesPalette)
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(True)

cv2.namedWindow('Semantic Segmentation Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('Semantic segmentation Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()




: 

## Test with RGB

In [4]:
def detect(image):
    with torch.no_grad():
        # HARDCODED SETTINGS
        weights = 'data/weights/yolopv2.pt'
        imgsz = 640
        device = '0'
        classes = None
        agnostic = False
        conf_thres = 0.3
        iou_thres=0.45

        # Load model
        stride = 32
        model  = torch.jit.load(weights)
        device = select_device(device)
        half = device.type != 'cpu'  # half precision only supported on CUDA
        model = model.to(device)

        if half:
            model.half()  # to FP16  
        model.eval()

        # Run inference
        if device.type != 'cpu':
            model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

        random_bgr = np.transpose(image, (1, 2, 0))

        img0 = cv2.resize(random_bgr, (1280,720), interpolation=cv2.INTER_LINEAR)
        img = letterbox(img0)[0]

        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0

        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        # Inference
        t1 = time_synchronized()
        [pred,anchor_grid],seg,ll= model(img)
        t2 = time_synchronized()

        # waste time: the incompatibility of  torch.jit.trace causes extra time consumption in demo version 
        # but this problem will not appear in offical version 
        tw1 = time_synchronized()
        pred = split_for_trace_model(pred,anchor_grid)
        tw2 = time_synchronized()

        # Apply NMS
        t3 = time_synchronized()
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=classes, agnostic=agnostic)
        t4 = time_synchronized()

        da_seg_mask = driving_area_mask(seg)
        ll_seg_mask = lane_line_mask(ll)

        show_seg_result(img0, (da_seg_mask,ll_seg_mask), is_demo=True)
        return img0

In [2]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=2), carla.Rotation(pitch=-10)), width=640, height=640):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((640, 640, 4), dtype=np.uint8)
video_output_seg = np.zeros((720, 1280, 3), dtype=np.uint8)
def camera_callback(image):
    global video_output
    global video_output_seg
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image_to_analyze = video_output[:, :, :3]
    image_to_analyze = np.transpose(image_to_analyze, (2, 0, 1))
    video_output_seg = detect(image_to_analyze)
camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(True)

#cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('RGB analyzed output', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
#        cv2.imshow('RGB Camera', video_output)
        cv2.imshow('RGB analyzed output', video_output_seg)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()

NameError: name 'spawn_vehicle' is not defined

In [6]:
import carla, time, pygame, math, random, cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from utils.utils import (
    time_synchronized, select_device, increment_path,
    scale_coords, xyxy2xywh, non_max_suppression, split_for_trace_model,
    driving_area_mask, lane_line_mask, plot_one_box, show_seg_result,
    AverageMeter, LoadImages, letterbox
)
from threading import Thread

# Load the model once
weights = 'data/weights/yolopv2.pt'
imgsz = 640
device = select_device('0')
model = torch.jit.load(weights).to(device)
half = device.type != 'cpu'  # half precision only supported on CUDA
if half:
    model.half()  # to FP16
model.eval()

# Run inference once to warm up the model
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

def detect(image):
    with torch.no_grad():
        random_bgr = np.transpose(image, (1, 2, 0))
        img0 = cv2.resize(random_bgr, (1280, 720), interpolation=cv2.INTER_LINEAR)
        img = letterbox(img0)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        # Inference
        t1 = time_synchronized()
        [pred, anchor_grid], seg, ll = model(img)
        t2 = time_synchronized()
        # Apply NMS
        pred = split_for_trace_model(pred, anchor_grid)
        pred = non_max_suppression(pred, 0.3, 0.45, classes=None, agnostic=False)
        da_seg_mask = driving_area_mask(seg)
        ll_seg_mask = lane_line_mask(ll)
        show_seg_result(img0, (da_seg_mask, ll_seg_mask), is_demo=True)
        return img0

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=2), carla.Rotation(pitch=-10)), width=640, height=640):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((640, 640, 4), dtype=np.uint8)
video_output_seg = np.zeros((720, 1280, 3), dtype=np.uint8)

def camera_callback(image):
    global video_output
    global video_output_seg
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image_to_analyze = video_output[:, :, :3]
    image_to_analyze = np.transpose(image_to_analyze, (2, 0, 1))
    video_output_seg = detect(image_to_analyze)

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(True)

cv2.namedWindow('RGB analyzed output', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB analyzed output', video_output_seg)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()

# Solution provided with threading

In [22]:
import carla, time, pygame, math, random, cv2
import numpy as np
import torch
from utils.utils import (
    time_synchronized, select_device, split_for_trace_model,
    non_max_suppression, driving_area_mask, lane_line_mask,
    show_seg_result, letterbox
)
from threading import Thread

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))

# Load the model once
weights = 'data/weights/yolopv2.pt'
imgsz = 640
device = select_device('0')
model = torch.jit.load(weights).to(device)
half = device.type != 'cpu'  # Half precision only supported on CUDA
if half:
    model.half()  # Convert to FP16
model.eval()

# Warm up model
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

def detect(image):
    with torch.no_grad():
        # Preprocess the image
        random_bgr = np.transpose(image, (1, 2, 0))
        img0 = cv2.resize(random_bgr, (1280, 720), interpolation=cv2.INTER_LINEAR)
        img = letterbox(img0)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # Convert to float16/32
        img /= 255.0  # Normalize to 0-1
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        [pred, anchor_grid], seg, ll = model(img)
        t2 = time_synchronized()

        # Apply segmentation masks
        pred = split_for_trace_model(pred, anchor_grid)
        pred = non_max_suppression(pred, 0.3, 0.45, classes=None, agnostic=False)
        da_seg_mask = driving_area_mask(seg)
        ll_seg_mask = lane_line_mask(ll)
        
        # Overlay segmentation masks on the original image
        mask = np.zeros_like(img0, dtype=np.uint8)
        mask[da_seg_mask == 1] = (0, 255, 0)  # Green for drivable area
        mask[ll_seg_mask == 1] = (0, 0, 255)  # Red for lane lines
        
        # Merge segmentation mask with the original image
        combined = cv2.addWeighted(img0, 0.7, mask, 0.3, 0)

        # Process the lane line mask for red line detection
        red_lane_mask = cv2.inRange(ll_seg_mask, 1, 255)

        # Clean up noise with morphological operationsq
        kernel = np.ones((5, 5), np.uint8)
        red_lane_mask = cv2.morphologyEx(red_lane_mask, cv2.MORPH_CLOSE, kernel)

        # Find contours of red lines
        contours, _ = cv2.findContours(red_lane_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw bounding boxes and alignment information
        red_boxes = [cv2.boundingRect(cnt) for cnt in contours]
        for (x, y, w, h) in red_boxes:
            cv2.rectangle(combined, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box

        # Lane alignment analysis
        if red_boxes:
            leftmost_red = min([x for x, y, w, h in red_boxes])
            rightmost_red = max([x + w for x, y, w, h in red_boxes])
            lane_center_x = (leftmost_red + rightmost_red) // 2
            img_center_x = combined.shape[1] // 2

            # Alignment status
            if abs(lane_center_x - img_center_x) < 20:
                alignment_status = "Car is CENTERED in the lane"
                print("Car is CENTERED in the lane")
            elif lane_center_x > img_center_x:
                alignment_status = "Car is CROSSING to the LEFT lane"
                print("Car is CROSSING to the LEFT lane")
            else:
                alignment_status = "Car is CROSSING to the RIGHT lane"
                print("Car is CROSSING to the RIGHT lane")

            # Draw alignment lines and text
            cv2.line(combined, (lane_center_x, 0), (lane_center_x, combined.shape[0]), (0, 0, 255), 2)
            cv2.line(combined, (img_center_x, 0), (img_center_x, combined.shape[0]), (255, 0, 0), 2)
            cv2.putText(combined, alignment_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Display segmentation mask for debugging
        cv2.imshow("Lane Line Mask", red_lane_mask)

        return combined



def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=2), carla.Rotation(pitch=-10)), width=640, height=640):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

counter = 2
vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((640, 640, 4), dtype=np.uint8)
video_output_seg = np.zeros((720, 1280, 3), dtype=np.uint8)

def camera_callback(image):
    global counter
    global video_output
    global video_output_seg
    counter -= 1
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    if counter == 0:
        image_to_analyze = video_output[:, :, :3]
        image_to_analyze = np.transpose(image_to_analyze, (2, 0, 1))
        Thread(target=run_inference, args=(image_to_analyze,)).start()

def run_inference(image_to_analyze):
    global counter
    global video_output_seg
    if counter == 0:
        video_output_seg = detect(image_to_analyze)
        counter = 2

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(False)

cv2.namedWindow('RGB analyzed output', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w:
                    vehicle.apply_control(carla.VehicleControl(throttle=1.0))
                elif event.key == pygame.K_s:
                    vehicle.apply_control(carla.VehicleControl(brake=1.0))
                elif event.key == pygame.K_a:
                    vehicle.apply_control(carla.VehicleControl(steer=-1.0))
                elif event.key == pygame.K_d:
                    vehicle.apply_control(carla.VehicleControl(steer=1.0))
            elif event.type == pygame.KEYUP:
                if event.key in [pygame.K_w, pygame.K_s]:
                    vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
                elif event.key in [pygame.K_a, pygame.K_d]:
                    vehicle.apply_control(carla.VehicleControl(steer=0.0))

        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        
        cv2.imshow('RGB analyzed output', video_output_seg)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()
    pygame.quit()


Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in the lane
Car is CENTERED in t

In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from utils.utils import (
    time_synchronized, select_device, increment_path,
    scale_coords, xyxy2xywh, non_max_suppression, split_for_trace_model,
    driving_area_mask, lane_line_mask, plot_one_box, show_seg_result,
    AverageMeter, LoadImages, letterbox
)
from threading import Thread

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))

# Load the model once
weights = 'data/weights/yolopv2.pt'
imgsz = 640
device = select_device('0')
model = torch.jit.load(weights).to(device)
half = device.type != 'cpu'  # half precision only supported on CUDA
if half:
    model.half()  # to FP16
model.eval()

# Run inference once to warm up the model
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

def detect(image):
    with torch.no_grad():
        # Original preprocessing and inference
        random_bgr = np.transpose(image, (1, 2, 0))
        img0 = cv2.resize(random_bgr, (1280, 720), interpolation=cv2.INTER_LINEAR)
        img = letterbox(img0)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        
        # Inference
        t1 = time_synchronized()
        [pred, anchor_grid], seg, ll = model(img)
        t2 = time_synchronized()
        
        # Apply NMS
        pred = split_for_trace_model(pred, anchor_grid)
        pred = non_max_suppression(pred, 0.3, 0.45, classes=None, agnostic=False)
        da_seg_mask = driving_area_mask(seg)
        ll_seg_mask = lane_line_mask(ll)
        
        # Visualize segmentation result
        show_seg_result(img0, (da_seg_mask, ll_seg_mask), is_demo=True)

        # NEW: Lane center calculation
        # Extract red lane markings from the lane line mask
        hsv = cv2.cvtColor(ll_seg_mask, cv2.COLOR_BGR2HSV)
        lower_red = np.array([0, 170, 110])  # Red lane threshold
        upper_red = np.array([10, 255, 255])
        red_mask = cv2.inRange(hsv, lower_red, upper_red)
        
        # Find contours of red lane markings
        contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        red_boxes = [cv2.boundingRect(cnt) for cnt in contours]
        
        # Draw bounding boxes on the original image
        for (x, y, w, h) in red_boxes:
            cv2.rectangle(img0, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Calculate leftmost and rightmost lane positions
        if red_boxes:
            leftmost_red = min([x for x, y, w, h in red_boxes])  # Smallest x value
            rightmost_red = max([x + w for x, y, w, h in red_boxes])  # Largest x+w value

            # Calculate the center of the lane
            lane_center_x = (leftmost_red + rightmost_red) // 2
            img_center_x = img0.shape[1] // 2  # Car's center assumed at image center

            # Determine alignment status
            if abs(lane_center_x - img_center_x) < 20:  # Threshold for being centered
                alignment_status = "Car is CENTERED in the lane"
            elif lane_center_x > img_center_x:
                alignment_status = "Car is CROSSING to the LEFT lane"
            else:
                alignment_status = "Car is CROSSING to the RIGHT lane"

            print(alignment_status)
            
            # Draw feedback on the image
            cv2.line(img0, (lane_center_x, 0), (lane_center_x, img0.shape[0]), (0, 0, 255), 2)
            cv2.line(img0, (img_center_x, 0), (img_center_x, img0.shape[0]), (255, 0, 0), 2)
            cv2.putText(img0, alignment_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Print the analysis mask
        cv2.imshow('Lane Analysis Mask', red_mask)

        return img0


def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=2), carla.Rotation(pitch=-10)), width=640, height=640):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera
counter = 2
vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((640, 640, 4), dtype=np.uint8)
video_output_seg = np.zeros((720, 1280, 3), dtype=np.uint8)

def camera_callback(image):
    global counter
    global video_output
    global video_output_seg
    counter = counter -1
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    if counter == 0:
        print("analyzing")
        image_to_analyze = video_output[:, :, :3]
        image_to_analyze = np.transpose(image_to_analyze, (2, 0, 1))
        # Start a new thread for the detect function
        Thread(target=run_inference, args=(image_to_analyze,)).start()

def run_inference(image_to_analyze):
    global counter
    global video_output_seg
    if counter == 0:
        video_output_seg = detect(image_to_analyze)
        #cv2.imwrite('output_image.png', video_output_seg)  # Save the image to disk
        counter = 2

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(False)

cv2.namedWindow('RGB analyzed output', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w:
                    vehicle.apply_control(carla.VehicleControl(throttle=1.0))
                elif event.key == pygame.K_s:
                    vehicle.apply_control(carla.VehicleControl(brake=1.0))
                elif event.key == pygame.K_a:
                    vehicle.apply_control(carla.VehicleControl(steer=-1.0))
                elif event.key == pygame.K_d:
                    vehicle.apply_control(carla.VehicleControl(steer=1.0))
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_w or event.key == pygame.K_s:
                    vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
                elif event key == pygame.K_a or event.key == pygame.K_d:
                    vehicle.apply_control(carla.VehicleControl(steer=0.0))

        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        
        cv2.imshow('RGB analyzed output', video_output_seg)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()
    pygame.quit()

In [5]:
import carla, time, pygame, math, random, cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from utils.utils import (
    time_synchronized, select_device, increment_path,
    scale_coords, xyxy2xywh, non_max_suppression, split_for_trace_model,
    driving_area_mask, lane_line_mask, plot_one_box, show_seg_result,
    AverageMeter, LoadImages, letterbox
)
from threading import Thread

# Load the model once
weights = 'data/weights/yolopv2.pt'
imgsz = 640
device = select_device('0')
model = torch.jit.load(weights).to(device)
half = device.type != 'cpu'  # half precision only supported on CUDA
if half:
    model.half()  # to FP16
model.eval()

# Run inference once to warm up the model
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

def detect(image):
    with torch.no_grad():
        # Crop the image to only the bottom part
        height, width = image.shape[1], image.shape[2]
        crop_height = height // 2  # Adjust this value to change the crop size
        cropped_image = image[:, crop_height:, :]  # Crop the bottom half of the image

        random_bgr = np.transpose(cropped_image, (1, 2, 0))
        img0 = cv2.resize(random_bgr, (1280, 720), interpolation=cv2.INTER_LINEAR)
        img = letterbox(img0)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        # Inference
        t1 = time_synchronized()
        _, seg, ll = model(img)
        t2 = time_synchronized()
        # Driving area and lane detection
        da_seg_mask = driving_area_mask(seg)
        ll_seg_mask = lane_line_mask(ll)
        show_seg_result(img0, (da_seg_mask, ll_seg_mask), is_demo=True)
        return img0

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=2), carla.Rotation(pitch=-10)), width=640, height=640):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((640, 640, 4), dtype=np.uint8)
video_output_seg = np.zeros((720, 1280, 3), dtype=np.uint8)

def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image_to_analyze = video_output[:, :, :3]
    image_to_analyze = np.transpose(image_to_analyze, (2, 0, 1))
    # Start a new thread for the detect function
    Thread(target=run_inference, args=(image_to_analyze,)).start()

def run_inference(image_to_analyze):
    global video_output_seg
    video_output_seg = detect(image_to_analyze)

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(True)

cv2.namedWindow('RGB analyzed output', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB analyzed output', video_output_seg)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()

c:\anaconda3\envs\carla-env\lib\site-packages\torch\nn\functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
